# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("./data/song_data")
song_files

['/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCEI128F424C983.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCIX128F4265903.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCUQ128E0783E2B.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCYE128F934CE1D.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCTK128F934B224.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCPZ128F4275C32.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCXB128F4286BD3.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCAJ12903CDFCC2.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCRU128F423F449.json',
 '/home/gykimura/Documents/udacity/git/song-play-analys

In [5]:
filepath = '/home/gykimura/Documents/udacity/git/song-play-analysis/data/song_data/A/B/C/TRABCIX128F4265903.json'

In [6]:
df_songs = pd.read_json(filepath, lines=True)
df_songs.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARNF6401187FB57032,40.79086,-73.96644,"New York, NY [Manhattan]",Sophie B. Hawkins,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,305.162,1994


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = df_songs[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
df_songs.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [10]:
artist_data = df_songs[['artist_id', 'title', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARNF6401187FB57032',
 'The Ballad Of Sleeping Beauty',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("./data/log_data")
log_files

['/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-26-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-09-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-30-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-01-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-05-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-19-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-03-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-18-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-20-events.json',
 '/home/gykimura/Documents/udacity/git/song-play-analys

In [13]:
filepath = '/home/gykimura/Documents/udacity/git/song-play-analysis/data/log_data/2018/11/2018-11-26-events.json'

In [14]:
df_logs = pd.read_json(filepath, lines=True)
df_logs.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Jordan,F,3,Hicks,259.26485,free,"Salinas, CA",PUT,NextSong,1.540009e+12,814,Supermassive Black Hole [Phones Control Voltag...,200,1543190563796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
1,None,Logged In,Jordan,F,4,Hicks,NaN,free,"Salinas, CA",PUT,Logout,1.540009e+12,814,None,307,1543190564796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
2,None,Logged Out,None,None,5,None,NaN,free,None,GET,Home,NaN,814,None,200,1543190570796,None,
3,None,Logged Out,None,None,6,None,NaN,free,None,PUT,Login,NaN,814,None,307,1543190571796,None,
4,None,Logged In,Jordan,F,7,Hicks,NaN,free,"Salinas, CA",GET,Home,1.540009e+12,814,None,200,1543190601796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
df_logs = df_logs[df_logs.page=="NextSong"]
df_logs.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Jordan,F,3,Hicks,259.26485,free,"Salinas, CA",PUT,NextSong,1.540009e+12,814,Supermassive Black Hole [Phones Control Voltag...,200,1543190563796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",37
6,Mobin Master Feat. Robin S.,Logged In,Jayden,M,1,Fox,428.64281,free,"New Orleans-Metairie, LA",PUT,NextSong,1.541034e+12,842,Show Me Love,200,1543193241796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
7,Duncan Dhu,Logged In,Bronson,M,0,Harris,211.90485,free,"Eugene, OR",PUT,NextSong,1.540907e+12,827,Rozando La Eternidad,200,1543193525796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",33
8,New Found Glory,Logged In,Bronson,M,1,Harris,222.69342,free,"Eugene, OR",PUT,NextSong,1.540907e+12,827,the king of wishful thinking,200,1543193736796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",33
9,Reel Big Fish,Logged In,Bronson,M,2,Harris,210.65098,free,"Eugene, OR",PUT,NextSong,1.540907e+12,827,Beer,200,1543193958796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",33


In [16]:
df_logs['ts'] = pd.to_datetime(df_logs['ts'], unit='ms')
df_logs['ts']

0     2018-11-26 00:02:43.796
6     2018-11-26 00:47:21.796
7     2018-11-26 00:52:05.796
8     2018-11-26 00:55:36.796
9     2018-11-26 00:59:18.796
                ...          
262   2018-11-26 18:58:10.796
263   2018-11-26 19:01:37.796
264   2018-11-26 19:04:31.796
266   2018-11-26 19:38:19.796
267   2018-11-26 19:41:55.796
Name: ts, Length: 216, dtype: datetime64[ns]

In [17]:
time_data = [df_logs['ts'], df_logs['ts'].dt.hour, df_logs['ts'].dt.day,
            df_logs['ts'].dt.weekofyear, df_logs['ts'].dt.month,
            df_logs['ts'].dt.year, df_logs['ts'].dt.weekday]
column_labels = ['ts','hour','day','week of year','month','year','weekday']

/tmp/ipykernel_60324/4162197901.py:2: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df_logs['ts'].dt.weekofyear, df_logs['ts'].dt.month,


In [18]:
time_data_dict = dict(zip(column_labels, time_data))
time_df = pd.DataFrame(time_data_dict)
time_df.head()

,ts,hour,day,week of year,month,year,weekday
0,2018-11-26 00:02:43.796,0,26,48,11,2018,0
6,2018-11-26 00:47:21.796,0,26,48,11,2018,0
7,2018-11-26 00:52:05.796,0,26,48,11,2018,0
8,2018-11-26 00:55:36.796,0,26,48,11,2018,0
9,2018-11-26 00:59:18.796,0,26,48,11,2018,0


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [19]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [20]:
df_logs.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [21]:
user_df = df_logs[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df

,userId,firstName,lastName,gender,level
0,37,Jordan,Hicks,F,free
6,101,Jayden,Fox,M,free
7,33,Bronson,Harris,M,free
8,33,Bronson,Harris,M,free
9,33,Bronson,Harris,M,free
...,...,...,...,...,...
262,80,Tegan,Levine,F,paid
263,80,Tegan,Levine,F,paid
264,80,Tegan,Levine,F,paid
266,52,Theodore,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [22]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [23]:
df_logs.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [24]:
#cur.execute("ROLLBACK")
#conn.commit()

In [25]:
for index, row in df_logs.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [26]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.